In [2]:
import numpy as np
import pandas as pd
import csv
import json
import io
from sklearn.cluster import KMeans

pd.set_option("display.max_rows",10)

try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [3]:
# Importing data set
filename = 'Traffic_data.csv'
traffic_data = pd.read_csv(filename)
traffic_data

C:\Program Files\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,YEAR,MONTH,HOUR,WEEKDAY
0,0,03/31/2017,0:00,NaN,NaN,40.645615,-73.909900,"(40.645615, -73.9099)",FOSTER AVENUE,NaN,...,3643404,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,2017,3,0,4
1,1,03/31/2017,0:00,NaN,NaN,40.762737,-73.839510,"(40.762737, -73.83951)",NaN,NaN,...,3643942,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN,2017,3,0,4
2,2,03/31/2017,0:00,BROOKLYN,11203,40.658478,-73.928180,"(40.658478, -73.92818)",EAST 53 STREET,WINTHROP STREET,...,3643587,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN,2017,3,0,4
3,3,03/31/2017,0:00,BROOKLYN,11235,40.580360,-73.967610,"(40.58036, -73.96761)",NEPTUNE AVENUE,OCEAN PARKWAY,...,3643847,NaN,NaN,NaN,NaN,NaN,2017,3,0,4
4,4,03/31/2017,0:00,NaN,NaN,40.845180,-73.914170,"(40.84518, -73.91417)",JEROME AVENUE,NaN,...,3642657,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2017,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007125,1007125,07/01/2012,9:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2896726,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,2012,7,9,6
1007126,1007126,07/01/2012,9:50,MANHATTAN,10019,40.766779,-73.996762,"(40.7667789, -73.9967618)",WEST SIDE HIGHWAY,WEST 50 STREET,...,37635,VAN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2012,7,9,6
1007127,1007127,07/01/2012,9:50,QUEENS,11001,40.736245,-73.702966,"(40.7362448, -73.7029656)",265 STREET,85 AVENUE,...,219395,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2012,7,9,6
1007128,1007128,07/01/2012,9:57,MANHATTAN,10065,40.765242,-73.957868,"(40.7652424, -73.9578679)",1 AVENUE,EAST 68 STREET,...,44907,PASSENGER VEHICLE,TAXI,NaN,NaN,NaN,2012,7,9,6


In [56]:
# Extracting coordinate data
coordinates = traffic_data[['LATITUDE','LONGITUDE','YEAR']].dropna(axis = 0, how = 'any')
focus_years = [2013,2014,2015,2016]

# Number of clusters to use
n_clusters = 4

# Initializing dictionaries to store data
cluster_data = {}
centroids = {}

for year in focus_years:
    data = coordinates.loc[coordinates['YEAR'].isin([year])]
    
    # Train KMeans on data
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data[['LATITUDE','LONGITUDE']])
    
    # Save the centroids to an array
    centroid = kmeans.cluster_centers_
    centroid_array = []
    for j in range(0,n_clusters):
        centroid_cord = {}
        centroid_cord['LATITUDE'] = centroid[j][0]
        centroid_cord['LONGITUDE'] = centroid[j][1]
        centroid_array.append(centroid_cord)
    centroids[str(year)] = centroid_array
    
    data['CLUSTER'] = kmeans.predict(data[['LATITUDE','LONGITUDE']])
    
    cluster_data[str(year)] = data[['LATITUDE','LONGITUDE','CLUSTER']].to_dict('list')

model_data = {}
model_data['centroids'] = centroids
model_data['datapoints'] = cluster_data

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
# Write JSON file with datapoints
with io.open('model_data.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(model_data,
                      indent=4, sort_keys=True,
                      separators=(',', ':'), ensure_ascii=False)
    outfile.write(to_unicode(str_))